# Chaines de Markhov

In [35]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import numpy as np
from scipy import stats
import pysal


table = pd.read_csv('Databases/swaphist.csv', delimiter = ";")
table['Date'] = pd.to_datetime(table['Date'],dayfirst=True)
#table = scipy.delete(table,table.shape[0]-1,0)
print(type(table))
print(np.array(table.quantile([k/5 for k in range(0, 5)])))
for k in range(0,5):
    print(k/5)

<class 'pandas.core.frame.DataFrame'>
[[ 11.2     9.8     8.8     9.4     8.3     7.5     7.5     7.6     6.5  ]
 [ 16.15   13.95   12.15   12.7    11.7    10.8    10.7    10.4     9.85 ]
 [ 19.85   16.15   13.9    14.8    13.8    12.85   12.7    12.65   11.8  ]
 [ 28.6    24.7    22.55   19.3    18.625  18.8    16.     16.65   17.5  ]
 [ 47.95   35.25   29.85   29.6    26.85   25.85   22.95   23.875  23.   ]]
0.0
0.2
0.4
0.6
0.8


# Définition des fonctions

### Calcul de la matrice de transition

In [62]:
#Calcule la place de l'élément dans un des n groupes
'''def place(elem,table,freq):
    xmin = min(table)
    xmax = max(table)
    delta = (xmax-xmin)/freq
    if table[elem]==xmax : freq-1
    else : return int((table[elem]-xmin)/delta)
    #return min(int((table[elem]-xmin)/delta),freq-1)'''

def place(elem, table, freq, quantiles):
    k = 0
    #print("elem : ",elem, " - valeur : ",table[elem], " - quantile : ", quantiles[k])
    #print("elem : ",elem)
    #print(table.size,table[elem])
    #print("table : ",table[elem])
    #print(" - quant : ",quantiles[k])
    while k<freq and table[elem]>=quantiles[k]:
        k += 1
    return k-1

def placement(table, freq):
    quantiles = np.array(table.quantile([k/freq for k in range(0, freq)]))
    liste = []
    for elem in table :
        k=0
        while k<freq and elem>=quantiles[k]:
            k += 1
        liste.append(k-1)
    return np.array(liste)

#Calcule la matrice de passage (fréquence de passage d'un groupe à l'autre)
def matrice(table, freq):
    #trans = np.zeros((freq,freq))
    trans = np.zeros((freq,freq))
    n = table.size
    places = placement(table, freq)
    #quantiles = np.array(table.quantile([k/freq for k in range(0, freq)]))
    for i in range(0,n-1):
        #trans[(place(i,table,freq, quantiles),place(i+1,table,freq, quantiles))] += 1
        trans[places[i],places[i+1]] += 1
    #for i in range(0,freq):
    for i in range(0,freq):
        sum_ligne = np.sum(trans[i])
        #Certains intervalles ne sont jamais visités. 
        if sum_ligne == 0 : 
            trans[(i,i)] = 0
        else :
            #for j in range(0,freq):
            for j in range(0,freq):
                trans[(i,j)] = trans[(i,j)]/sum_ligne
    return trans



#La matrice de transition est-elle bien stochastique ?
test = matrice(table['EUPG15'],20)
print(test)
for i in range(0,test.shape[0]):
    #print(sum(test[i]))
    if(sum(test[i])!=1.0): print(sum(test[i]))


colors = ['RED','GREEN','BLUE','BLACK','ORANGE','PINK','BROWN','YELLOW','CYAN']
#fig, ax = plt.subplots()
#ax.plot_date(table['Date'], table['1to5'],'-',color='BLACK')
#ax.plot_date(table['Date'],20,'-',color='RED')
#fig.autofmt_xdate()

[[ 0.86486486  0.13513514  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.13888889  0.63888889  0.16666667  0.02777778  0.02777778  0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.21052632  0.55263158  0.21052632  0.02631579  0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.24324324  0.62162162  0.13513514  0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.02777778  0.13888889  0.58333333  0.16666667
  

Les matrices de transitions ne sont pas stochastiques car certains intervalles de valeurs ne sont jamais visités : typiquement, les valeurs élevés atteintes durant le pic de 2008-2013. Les variations sont très importantes lors de cette période et les volatilités font des sauts importants d'une semaine à l'autre.

Il faudra prendre cela en compte lorsque l'on conduira les tests. En effet, les tests doivent être effectués sur des matrices stochastiques. On supprimera donc les colonnes et lignes vides.

### Fonctions auxiliaires pour le calcul des statistiques de test

In [41]:
#Une fonction pour supprimer les colonnes et lignes qui ne contiennent que des zeros
def zero(mat):
    liste = []
    for i in range(0,mat.shape[0]-1):
        if(sum(mat[i])==0): liste.append(i)
    mat = scipy.delete(mat,liste,0)
    liste = []
    for i in range(0,mat.shape[1]-1): 
        if(sum(mat[:,i])==0): liste.append(i)
    mat = scipy.delete(mat,liste,1)
    return mat

In [42]:
def A(table,i,freq,T):
    a=0
    xmin = min(table)
    xmax = max(table)
    delta = (xmax-xmin)/T
    tmp=0
    for t in range(0,T):
        sous_table = table[int(t*delta+xmin): int((t+1)*delta+xmin)]
        sous_table.index = range(sous_table.shape[0])
        #tmp =+ n(sous_table,i,freq)
        if n(sous_table,i,freq)!=0 : a+=1
    return a

def B(MT,i):
    B = []
    for j in range(0,MT.shape[1]):
        if MT[i,j]!=0 : B.append(j)
    return B

def n(sous_table,i,freq):
    tot = 0
    quantiles = np.array(sous_table.quantile([k/freq for k in range(0, freq)]))
    for k in range(0,sous_table.size) :
        if place(k,sous_table,freq,quantiles)==i : tot += 1
    return tot

def stat(table, MT, sous_table, sous_MT, freq):
    dg_lbt = 0
    stat = 0
    for i in range(0,MT.shape[0]):
        #print("i : ",i)
        ni = n(sous_table,i, freq)
        Bi = B(MT,i)
        for j in Bi:
            stat += ni*(sous_MT[i,j]-MT[i,j])**2/MT[i,j]
            #if MT[i,j]!=0 : stat += ni*(sous_MT[i,j]-MT[i,j])**2/MT[i,j]
    return stat

In [43]:
#Les matrices de transition de la table est-elle homogène ?
def homog(table, freq=50, T=5):
    for i,elem in enumerate(table):
        if i>0:
            Q,p = homog_temp(table[elem].values,freq,T)
            m1 = homog_int(table[elem].values,freq,T)
            m2 = homog_reg(table[elem].values,freq,T)
            print("p-valeur : ",p, " - résultat du test par intervalles : ",m1," - résultat du test par régions : ",m2)
            rep = p>0.05 and m1>0.5 and m2>0.5
            if rep : print("La série de valeur pour le swaption ",elem," est homogène")
            else : print("La série de valeur pour le swaption ",elem," n'est pas homogène")
    return 0

## Les matrices de transition sont elles bien homogènes ?

Pour que la matrice de transition calculée à partir des fréquences de transition ait un sens, il fait s'assurer qu'elle est bien homogène en temps. On va donc mener plusieurs test pour vérifier cette hypothèse lourde.

### Test d'homogénéité des intervalles par rapport à la matrice de transition globale

On subdivise notre série de valeurs en un nombre $T$ d'intervalles. On vérifie que, si on calcule la matrice de transition sur chacun des intervalles, on retrouve bien la matrice de transition calculée sur toutes les valeurs de la série par un test du khi-2.


Notre hypothèse nulle est donc : $H_0 = \{\forall t, \le T, \forall i,j, p_{i,j} (t) = p_{i,j}\}$

In [44]:
def homog_temp(table, freq, T=5):
    N = table.size
    Q = 0
    delta = N/T
    MT = matrice(table, freq)
    for t in range(0,T):
        sous_table = table[int(t*delta): int((t+1)*delta)]
        sous_table.index = range(sous_table.shape[0])
        sous_MT = matrice(sous_table, freq)
        Q += stat(table,MT,sous_table,sous_MT,freq)
    
    dg_lib = 0
    MT = matrice(table, freq)
    for i in range(0, MT.shape[0]-1):
        if A(table,i,freq,T)>0 : dg_lib += (len(B(MT,i))-1)*(A(table,i,freq,T)-1)
    print("Degrés de liberté : ",dg_lib)
    return Q, 1-stats.chi2.cdf(Q,dg_lib)

### Test d'homogénéité par intervalle

On subdivise notre série de valeurs en un nombre $T$ d'intervalles. On va effectuer $T$ tests pour vérifier si chacune des $T$ matrices de transitions sont bien les mêmes que la matrice de transition globale.


Notre hypothèse nulle est donc, pour chacun des intervalles $ 0\le t <T$ : $H_0^t = \{\forall i,j, p_{i,j} (t) = p_{i,j}\}$

In [74]:
def homog_int(table, freq, T=5):
    N = table.size
    Q = 0
    delta = N/T
    moy = 0
    MT = matrice(table, freq)
    for t in range(0,T):
        sous_table = table[int(t*delta): int((t+1)*delta)]
        sous_table.index = range(sous_table.shape[0])
        sous_MT = matrice(sous_table, freq)
        Q = stat(table,MT,sous_table,sous_MT,freq)
        dg_lib = 0
        MT = matrice(table, freq)
        for i in range(0, MT.shape[0]-1):
            #print(B(MT,i),A(table,i,freq,T))
            dg_lib += (len(B(MT,i))-1)*(A(table,i,freq,T)-1)
        p = 1-stats.chi2.cdf(Q,dg_lib)
        print("Q : ",Q, " - dg de lbte : ",dg_lib)
        print("p-valeur du test sur le ",t,"-ième intervalle : ",p)
        if p>0.05 : moy += 1
    
    print(moy," intervalles de temps sur ",T," sont homogènes")    
    return moy/T

### Test région contre région

On subdivise toujours notre série de valeurs en un nombre $T$ d'intervalles. Pour finir, on vérifie si les matrices de transitions de deux périodes sont identiques.

Notre hypothèse nulle est donc, pour chacun des intervalles $ 0\le t,u <T$ : $H_0^{t,u} = \{\forall i,j, p_{i,j}^t (t) = p_{i,j}^u\}$

In [68]:
def homog_reg(table, freq, T=5):
    N = table.size
    Q = 0
    delta = N/T
    moy = 0
    t,u = 0,0
    for t in range(0,T):
        for u in range(t+1,T):
            sous_table1 = table[int(t*delta): int((t+1)*delta)]
            sous_table2 = table[int(u*delta): int((u+1)*delta)]
            sous_table1.index = range(int((t+1)*delta)-int(t*delta))
            sous_table2.index = range(int((u+1)*delta)-int(u*delta))
            sous_MT1 = matrice(sous_table1, freq)
            sous_MT2 = matrice(sous_table2, freq)
            Q = stat(sous_table1,sous_MT1,sous_table2,sous_MT2,freq)
            dg_lib = 0
            MT = matrice(sous_table1, freq)
            for i in range(0, MT.shape[0]-1):
                dg_lib += max((len(B(MT,i))-1)*(A(table,i,freq,T)-1),0)
            p = 1-stats.chi2.cdf(Q,dg_lib)
            print("Q : ",Q," - dg lib : ",dg_lib)
            print("p-valeur du test entre le ",t+1,"-ième et le ",u+1,"-ième intervalle : ",p)
            if p>0.05 : moy += 1
    
    print(moy," paires d'intervalles de temps sur ",T," intervalles sont homogènes")    
    return 2*moy/(T*(T-1))

## Deux matrices de transition sont-elles égales ?

Maintenant, on peut tester si deux matrices de transition sont égales. On appliquera la statistique de test précédement définie aux deux matrices.

Notre hypothèse nulle est donc, si on note $(p_{i,j}^0)$ et $(p_{i,j}^1)$ les coefficents de chacune des deux matrices de transition : $H_0 = \{\forall i,j, p_{i,j}^0 = p_{i,j}^1\}$

In [77]:
def egales(table1, table2, freq=10):
    MT1 = matrice(table1, freq)
    MT2 = matrice(table2, freq)
    Q = stat(table1,MT1,table2,MT2,freq)
    
    dg_lib = 0
    for i in range(0, MT1.shape[0]-1):
        dg_lib += (len(B(MT1,i))-1)
    return Q, 1-stats.chi2.cdf(Q,dg_lib)


# Tests sur toute la période 2001-2016

## Homogénéité

In [64]:
homog_temp(table['EUPG15'],10)

Degrés de liberté :  80


(762.55934408897508, 0.0)

In [65]:
homog_int(table['EUPG15'],10,T=5)

[0, 1, 2] 5
[0, 1, 2] 5
[1, 2, 3, 4] 5
[2, 3, 4] 5
[3, 4, 5, 6] 5
[4, 5, 6] 5
[5, 6, 7] 5
[6, 7, 8] 5
[7, 8, 9] 5
Q :  137.336482394  - dg de lbte :  80
p-valeur du test sur le  0 -ième intervalle :  7.07176108886e-05
[0, 1, 2] 5
[0, 1, 2] 5
[1, 2, 3, 4] 5
[2, 3, 4] 5
[3, 4, 5, 6] 5
[4, 5, 6] 5
[5, 6, 7] 5
[6, 7, 8] 5
[7, 8, 9] 5
Q :  73.5023802989  - dg de lbte :  80
p-valeur du test sur le  1 -ième intervalle :  0.682673023804
[0, 1, 2] 5
[0, 1, 2] 5
[1, 2, 3, 4] 5
[2, 3, 4] 5
[3, 4, 5, 6] 5
[4, 5, 6] 5
[5, 6, 7] 5
[6, 7, 8] 5
[7, 8, 9] 5
Q :  179.403438622  - dg de lbte :  80
p-valeur du test sur le  2 -ième intervalle :  1.36245947857e-09
[0, 1, 2] 5
[0, 1, 2] 5
[1, 2, 3, 4] 5
[2, 3, 4] 5
[3, 4, 5, 6] 5
[4, 5, 6] 5
[5, 6, 7] 5
[6, 7, 8] 5
[7, 8, 9] 5
Q :  211.212381929  - dg de lbte :  80
p-valeur du test sur le  3 -ième intervalle :  8.84847750626e-14
[0, 1, 2] 5
[0, 1, 2] 5
[1, 2, 3, 4] 5
[2, 3, 4] 5
[3, 4, 5, 6] 5
[4, 5, 6] 5
[5, 6, 7] 5
[6, 7, 8] 5
[7, 8, 9] 5
Q :  161.10466084

0.2

In [69]:
homog_reg(table['EUPG15'],10)

Q :  41.4570281901  - dg lib :  96
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.999999784421
Q :  37.4914858653  - dg lib :  96
p-valeur du test entre le  1 -ième et le  3 -ième intervalle :  0.999999988237
Q :  41.5928597196  - dg lib :  96
p-valeur du test entre le  1 -ième et le  4 -ième intervalle :  0.99999976379
Q :  35.1490094826  - dg lib :  96
p-valeur du test entre le  1 -ième et le  5 -ième intervalle :  0.999999998347
Q :  69.4827882435  - dg lib :  92
p-valeur du test entre le  2 -ième et le  3 -ième intervalle :  0.961515485846
Q :  48.2663926682  - dg lib :  92
p-valeur du test entre le  2 -ième et le  4 -ième intervalle :  0.999951829433
Q :  80.7038435374  - dg lib :  92
p-valeur du test entre le  2 -ième et le  5 -ième intervalle :  0.793841676308
Q :  42.6491609977  - dg lib :  96
p-valeur du test entre le  3 -ième et le  4 -ième intervalle :  0.999999527334
Q :  71.1801831502  - dg lib :  96
p-valeur du test entre le  3 -ième et le  5 -ième inte

1.0

Comme on pouvait s'y attendre, les matrices de transition sur la période 2001-2016 ne sont homogène dans le temps pour aucun des swaptions. Nous allons donc restreindre notre études à de plus petits intervalles.

# Test sur la période 2001-2008

In [51]:
tab1 = table[0:302]

In [70]:
homog_reg(tab1['EUPG15'],10)

Q :  27.5  - dg lib :  14
p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.0165640564272
Q :  28.6206349206  - dg lib :  14
p-valeur du test entre le  1 -ième et le  3 -ième intervalle :  0.011756303291
Q :  27.5888095238  - dg lib :  14
p-valeur du test entre le  1 -ième et le  4 -ième intervalle :  0.016124579416
Q :  25.3080357143  - dg lib :  14
p-valeur du test entre le  1 -ième et le  5 -ième intervalle :  0.0316454063589
Q :  32.9728835979  - dg lib :  10
p-valeur du test entre le  2 -ième et le  3 -ième intervalle :  0.000275258597266
Q :  29.6792063492  - dg lib :  10
p-valeur du test entre le  2 -ième et le  4 -ième intervalle :  0.000966463426443
Q :  24.9538690476  - dg lib :  10
p-valeur du test entre le  2 -ième et le  5 -ième intervalle :  0.00543363462543
Q :  37.6271428571  - dg lib :  14
p-valeur du test entre le  3 -ième et le  4 -ième intervalle :  0.000592443188751
Q :  38.8333333333  - dg lib :  14
p-valeur du test entre le  3 -ième et le  5 -ième

0.0

In [71]:
homog_int(tab1['EUPG15'],20)

[0, 1, 2, 3] 5
[0, 1, 2, 3] 0
[0, 1, 2, 3, 4, 8] 0
[1, 2, 3, 4, 8] 0
[3, 4, 5] 0
[4, 5, 6, 9] 0
[5, 6, 7] 0
[5, 6, 7, 8, 9] 0
[5, 6, 7, 8, 9] 0
[7, 8, 9, 10, 12] 0
[9, 10, 11, 12] 3
[10, 11, 12, 13, 16] 0
[9, 10, 11, 12, 13, 14, 17] 0
[12, 13, 14, 15, 16] 0
[10, 13, 14, 15, 16, 18] 0
[12, 13, 14, 15, 16, 17, 18] 0
[13, 14, 15, 16, 17, 18] 0
[14, 15, 16, 17, 19] 0
[14, 16, 17, 18, 19] 0
Q :  68.2411904762  - dg de lbte :  -51
p-valeur du test sur le  0 -ième intervalle :  nan
[0, 1, 2, 3] 5
[0, 1, 2, 3] 0
[0, 1, 2, 3, 4, 8] 0
[1, 2, 3, 4, 8] 0
[3, 4, 5] 0
[4, 5, 6, 9] 0
[5, 6, 7] 0
[5, 6, 7, 8, 9] 0
[5, 6, 7, 8, 9] 0
[7, 8, 9, 10, 12] 0
[9, 10, 11, 12] 3
[10, 11, 12, 13, 16] 0
[9, 10, 11, 12, 13, 14, 17] 0
[12, 13, 14, 15, 16] 0
[10, 13, 14, 15, 16, 18] 0
[12, 13, 14, 15, 16, 17, 18] 0
[13, 14, 15, 16, 17, 18] 0
[14, 15, 16, 17, 19] 0
[14, 16, 17, 18, 19] 0
Q :  75.4691798942  - dg de lbte :  -51
p-valeur du test sur le  1 -ième intervalle :  nan
[0, 1, 2, 3] 5
[0, 1, 2, 3] 0
[0, 1, 2, 

0.0

In [55]:
homog_temp(tab1['EUPG15'],10,T=5)

Degrés de liberté :  8


(80.117248692091849, 4.6296300126869028e-14)

## Test sur la période 2008-2014

In [17]:
tab2 = table[302:615]
tab2.index = range(tab2.shape[0])

In [56]:
homog_reg(tab2['EUPG15'],20)

p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  0.999474512733
p-valeur du test entre le  1 -ième et le  3 -ième intervalle :  0.999993430696
p-valeur du test entre le  1 -ième et le  4 -ième intervalle :  0.999990146217
p-valeur du test entre le  1 -ième et le  5 -ième intervalle :  0.950438000462
p-valeur du test entre le  2 -ième et le  3 -ième intervalle :  0.999925493271
p-valeur du test entre le  2 -ième et le  4 -ième intervalle :  0.999826464865
p-valeur du test entre le  2 -ième et le  5 -ième intervalle :  0.99973386904
p-valeur du test entre le  3 -ième et le  4 -ième intervalle :  0.999298285833
p-valeur du test entre le  3 -ième et le  5 -ième intervalle :  0.874418999691
p-valeur du test entre le  4 -ième et le  5 -ième intervalle :  0.575029154141
10  paires d'intervalles de temps sur  5  intervalles sont homogènes


1.0

In [20]:
import pysal
q5 = pysal.Quantiles(table['EUPG15']).yb

In [78]:
egales(tab1['EUPG15'],tab1['EUPG110'])

(82.787312201168447, 1.4181538510182889e-07)

In [79]:
egales(tab1['EUPG15'],tab1['EUPG1020'])

(128.09824627058561, 4.2188474935755949e-15)

In [80]:
egales(tab1['EUPG55'],tab1['EUPG110'])

(76.000065374172422, 7.3049121787072124e-06)